<a href="https://colab.research.google.com/github/kochol/tf_tests/blob/master/test01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Test 01

This is the first test of my tf code.

In [1]:
import tensorflow as tf
from tensorflow.keras import layers

# enable tf eager execution to support iterating
tf.enable_eager_execution()

print(tf.VERSION)
print(tf.keras.__version__)

1.13.1
2.2.4-tf


## Create the model

Now create the model

In [2]:
inputs = tf.keras.Input(shape=(7,))  # Returns a placeholder tensor

# A layer instance is callable on a tensor, and returns a tensor.
x = layers.Dense(64, activation='relu')(inputs)
x = layers.Dense(64, activation='relu')(x)
predictions = layers.Dense(1)(x)

model = tf.keras.Model(inputs=inputs, outputs=predictions)

# I want to calc on float numbers so I use mse and mae for loss and metrics
# Also I have to find out which optimizer is good for me.
model.compile(optimizer=tf.train.RMSPropOptimizer(0.001),
              loss='mse',
              metrics=['mae', 'mse'])

model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 7)                 0         
_________________________________________________________________
dense (Dense)                (None, 64)                512       
_________________________________________________________________
dense_1 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 65        
Total params: 4,737
Trainable params: 4,737
Non-trainable params: 0
_________________________________________________________________


## Load data from csv file

In [3]:
import pandas as pd
import numpy as np

# Download mydata.csv from github
!curl --remote-name \
     -H 'Accept: application/vnd.github.v3.raw' \
     --location https://raw.githubusercontent.com/kochol/tf_tests/master/mydata.csv

# load all data
mydata = pd.read_csv("mydata.csv")

print(mydata.columns)
print(mydata.dtypes)
print(mydata.shape)

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  154k  100  154k    0     0  2082k      0 --:--:-- --:--:-- --:--:-- 2082k
Index(['start', 'open', 'high', 'low', 'close', 'vwp', 'volume', 'trades'], dtype='object')
start       int64
open      float64
high      float64
low       float64
close     float64
vwp       float64
volume    float64
trades      int64
dtype: object
(1884, 8)


Prepare the data for training

In [4]:
train = pd.DataFrame(
    {
        'open': mydata[0:-1].open.astype('int32'),
        'high': mydata[0:-1].high.astype('int32'),
        'low': mydata[0:-1].low.astype('int32'),
        'close': mydata[0:-1].close.astype('int32'),
        'vwp': mydata[0:-1].vwp.astype('int32'),
        'volume': mydata[0:-1].volume.astype('int32'),
        'trades': mydata[0:-1].trades.astype('int32')
    })
print(train.columns)
print(train.dtypes)
print(train.shape)

test = pd.DataFrame({'price': mydata[1:].close.astype('int32')})
print(test.columns)
print(test.dtypes)
print(test.shape)

Index(['open', 'high', 'low', 'close', 'vwp', 'volume', 'trades'], dtype='object')
open      int32
high      int32
low       int32
close     int32
vwp       int32
volume    int32
trades    int32
dtype: object
(1883, 7)
Index(['price'], dtype='object')
price    int32
dtype: object
(1883, 1)


## Train the model

In [5]:
train_np = train.to_numpy()
test_np = test.to_numpy()
model.fit(train_np, test_np, epochs=1000, batch_size=32)#, validation_split = 0.2)

Instructions for updating:
Use tf.cast instead.
Epoch 1/1000
1883/1883 [==============================] - 0s 75us/sample - loss: 374208.0448 - mean_absolute_error: 246.4934 - mean_squared_error: 374208.1250
Epoch 2/1000
1883/1883 [==============================] - 0s 40us/sample - loss: 39487.6009 - mean_absolute_error: 139.5270 - mean_squared_error: 39487.6016
Epoch 3/1000
1883/1883 [==============================] - 0s 43us/sample - loss: 31978.8475 - mean_absolute_error: 106.9229 - mean_squared_error: 31978.8496
Epoch 4/1000
1883/1883 [==============================] - 0s 42us/sample - loss: 27257.6070 - mean_absolute_error: 129.4616 - mean_squared_error: 27257.6055
Epoch 5/1000
1883/1883 [==============================] - 0s 42us/sample - loss: 27824.8242 - mean_absolute_error: 112.0235 - mean_squared_error: 27824.8223
Epoch 6/1000
1883/1883 [==============================] - 0s 40us/sample - loss: 23758.6615 - mean_absolute_error: 119.7760 - mean_squared_error: 23758.6602
Epoch 7/

In [11]:
example_batch = train_np[-100:]
example_result = model.predict(example_batch)
print(example_result)
print(test_np[-100:])

[[5594.9683]
 [5572.1465]
 [5578.233 ]
 [5579.611 ]
 [5595.1562]
 [5567.0703]
 [5568.8584]
 [5579.8916]
 [5601.448 ]
 [5612.5957]
 [5620.79  ]
 [5605.768 ]
 [5603.794 ]
 [5632.5337]
 [5640.927 ]
 [5635.503 ]
 [5620.089 ]
 [5612.453 ]
 [5586.689 ]
 [5588.601 ]
 [5575.5933]
 [5573.264 ]
 [5578.9546]
 [5600.1865]
 [5620.317 ]
 [5624.8267]
 [5636.888 ]
 [5609.5576]
 [5547.2666]
 [5512.569 ]
 [5491.6235]
 [5476.213 ]
 [5476.588 ]
 [5486.145 ]
 [5485.84  ]
 [5503.3477]
 [5479.9316]
 [5464.7236]
 [5443.7153]
 [5455.211 ]
 [5475.153 ]
 [5469.327 ]
 [5472.2915]
 [5469.749 ]
 [5486.4355]
 [5492.9893]
 [5482.698 ]
 [5469.657 ]
 [5477.38  ]
 [5469.6934]
 [5456.8477]
 [5455.5576]
 [5469.92  ]
 [5455.46  ]
 [5477.644 ]
 [5492.5054]
 [5497.4746]
 [5494.0464]
 [5490.185 ]
 [5496.7466]
 [5505.1396]
 [5532.739 ]
 [5539.2227]
 [5542.6636]
 [5531.437 ]
 [5519.3125]
 [5370.388 ]
 [5304.515 ]
 [5288.508 ]
 [5374.297 ]
 [5483.503 ]
 [5485.51  ]
 [5420.801 ]
 [5406.68  ]
 [5378.244 ]
 [5360.6816]
 [5371.2983]